# Data Retreival and Aggregation

Dependent Variable (ADP): We will use boto3 package to get the historical daily inmates in custody data from our secure s3 bucket. *Note: you will not be able to retrieve this data unless you are a part of the DCJ organization and have configured your AWS credentials for your compputer.*

**If you are attempting to clone this repository but do not have access to our AWS data warehouse, SKIP this notebook and use the static data in the `/Data` folder of this project.**

Independent Variables: We will also use the SoSQL language to grab exogenous variables from NYC Open Data Portal. 

Both datasets will then be aggregated to 30-day rolling averages or counts to be used in the analysis.

In [1]:
#import packages
import pandas as pd
import numpy as np
from functions import *
%matplotlib inline

### Step 1: Get Target Variable

In [2]:
#specify s3 bucket and file locations
bucket_name = 'doc-daily-inmates-in-custody'
folder_name = 'data/merged_files/'
file_name = 'agg_daily_pop.csv'

#get the daily merged file from s3 bucket
adp_df = get_file(bucket_name, folder_name, file_name)
adp_df.head()

Trying to get object from bucket
Got file contents from AWS S3
Saved file contents as dataframe


,snapshot_date,race_A,race_B,race_I,race_O,race_U,race_W,gender_F,gender_Gender Unknown,gender_M,...,inmate_status_code_CSP,inmate_status_code_DE,inmate_status_code_DEP,inmate_status_code_DNS,inmate_status_code_DPV,inmate_status_code_SCO,inmate_status_code_SSR,Total Population,snapshot_month,snapshot_year
0,2016-06-02,158,5603,9,2706,108,1252,614,16,9206,...,63,6892,821,98,573,43.0,163,9836,6,2016
1,2016-06-03,157,5584,9,2676,106,1248,610,13,9157,...,63,6860,807,91,577,43.0,180,9780,6,2016
2,2016-06-04,159,5581,9,2661,112,1243,616,17,9132,...,63,6829,792,100,592,44.0,179,9765,6,2016
3,2016-06-05,161,5661,9,2692,108,1263,629,13,9252,...,64,6928,796,100,591,44.0,179,9894,6,2016
4,2016-06-06,158,5674,9,2702,107,1254,633,14,9257,...,65,6983,798,85,572,47.0,178,9904,6,2016


In [3]:
#aggregate to 30 day intervals
#set data type
adp_df['snapshot_date'] = pd.to_datetime(adp_df['snapshot_date'])
#get latest snapshot date
max_date = adp_df['snapshot_date'].max()
#aggregate to 30 day periods
_30day_interval_data = adp_df.resample('30D', on='snapshot_date',origin = max_date,closed='right',label = 'right').mean().round().fillna(0).reset_index()
_30day_interval_data = _30day_interval_data.rename(columns = {'snapshot_date':'End Date','Total Population':'ADP'})
_30day_interval_data['Start Date'] = _30day_interval_data['End Date'] - pd.to_timedelta(29, unit='D')

_30day_interval_data.head()

,End Date,race_A,race_B,race_I,race_O,race_U,race_W,gender_F,gender_Gender Unknown,gender_M,...,inmate_status_code_DE,inmate_status_code_DEP,inmate_status_code_DNS,inmate_status_code_DPV,inmate_status_code_SCO,inmate_status_code_SSR,ADP,snapshot_month,snapshot_year,Start Date
0,2016-06-13,155.0,5613.0,9.0,2691.0,108.0,1243.0,626.0,14.0,9180.0,...,6919.0,800.0,90.0,564.0,41.0,166.0,9820.0,6.0,2016.0,2016-05-15
1,2016-07-13,151.0,5605.0,8.0,2693.0,110.0,1230.0,620.0,13.0,9163.0,...,6867.0,798.0,102.0,571.0,44.0,154.0,9796.0,6.0,2016.0,2016-06-14
2,2016-08-12,150.0,5539.0,9.0,2723.0,95.0,1214.0,617.0,14.0,9099.0,...,6804.0,800.0,110.0,562.0,41.0,139.0,9730.0,7.0,2016.0,2016-07-14
3,2016-09-11,159.0,5587.0,8.0,2781.0,97.0,1192.0,659.0,13.0,9152.0,...,6910.0,824.0,94.0,580.0,31.0,110.0,9824.0,8.0,2016.0,2016-08-13
4,2016-10-11,165.0,5599.0,5.0,2834.0,91.0,1151.0,651.0,9.0,9186.0,...,6908.0,838.0,108.0,560.0,37.0,127.0,9845.0,9.0,2016.0,2016-09-12


In [4]:
#also do 7 day rolling averages ~ weekly
#aggregate to 30 day periods
_7day_interval_data = adp_df.resample('7D', on='snapshot_date',origin = max_date,closed='right',label = 'right').mean().round().fillna(0).reset_index()
_7day_interval_data = _7day_interval_data.rename(columns = {'snapshot_date':'End Date','Total Population':'ADP'})
_7day_interval_data['Start Date'] = _7day_interval_data['End Date'] - pd.to_timedelta(6, unit='D')

_7day_interval_data.head()

,End Date,race_A,race_B,race_I,race_O,race_U,race_W,gender_F,gender_Gender Unknown,gender_M,...,inmate_status_code_DE,inmate_status_code_DEP,inmate_status_code_DNS,inmate_status_code_DPV,inmate_status_code_SCO,inmate_status_code_SSR,ADP,snapshot_month,snapshot_year,Start Date
0,2016-06-05,159.0,5607.0,9.0,2684.0,108.0,1252.0,617.0,15.0,9187.0,...,6877.0,804.0,97.0,583.0,44.0,175.0,9819.0,6.0,2016.0,2016-05-30
1,2016-06-12,154.0,5610.0,9.0,2692.0,108.0,1238.0,629.0,13.0,9168.0,...,6929.0,798.0,88.0,556.0,40.0,164.0,9811.0,6.0,2016.0,2016-06-06
2,2016-06-19,149.0,5607.0,9.0,2695.0,109.0,1231.0,624.0,12.0,9165.0,...,6893.0,801.0,86.0,548.0,41.0,172.0,9801.0,6.0,2016.0,2016-06-13
3,2016-06-26,156.0,5617.0,7.0,2668.0,118.0,1240.0,624.0,14.0,9168.0,...,6864.0,799.0,105.0,572.0,44.0,142.0,9805.0,6.0,2016.0,2016-06-20
4,2016-07-03,154.0,5610.0,7.0,2689.0,113.0,1231.0,629.0,12.0,9162.0,...,6858.0,794.0,110.0,584.0,49.0,145.0,9804.0,6.0,2016.0,2016-06-27


In [5]:
_7day_pop_tot = _7day_interval_data[['Start Date','End Date','ADP']]
# for the weekly data, we are going to double check for any missing values and do
# a linear interpolation just in case. 
_7day_pop_tot = _7day_pop_tot.replace(0,np.nan)
_7day_pop_tot["ADP"] = _7day_pop_tot["ADP"].interpolate(method='linear', axis=0).ffill().bfill()


_7day_pop_tot.tail()

,Start Date,End Date,ADP
430,2024-08-26,2024-09-01,6468.0
431,2024-09-02,2024-09-08,6512.0
432,2024-09-09,2024-09-15,6508.0
433,2024-09-16,2024-09-22,6507.0
434,2024-09-23,2024-09-29,6527.0


In [6]:
#lets only look at the total population
_30day_pop_tot = _30day_interval_data[['Start Date','End Date','ADP']]
daily_pop = adp_df[['snapshot_date','Total Population']]
#save to csv to use in other scripts
daily_pop.to_csv("../Data/daily_pop.csv")
_7day_pop_tot.to_csv("../Data/_7_day_adp.csv")
_30day_pop_tot.to_csv("../Data/_30_day_adp.csv")

In [7]:
daily_pop.head()

,snapshot_date,Total Population
0,2016-06-02,9836
1,2016-06-03,9780
2,2016-06-04,9765
3,2016-06-05,9894
4,2016-06-06,9904


In [8]:
_30day_pop_tot.dtypes

Start Date    datetime64[ns]
End Date      datetime64[ns]
ADP                  float64
dtype: object

In [9]:
_30day_pop_tot.tail()

,Start Date,End Date,ADP
97,2024-05-03,2024-06-01,6313.0
98,2024-06-02,2024-07-01,6383.0
99,2024-07-02,2024-07-31,6387.0
100,2024-08-01,2024-08-30,6443.0
101,2024-08-31,2024-09-29,6509.0


### Step 2: Get IVs

* length of stay

* 30-day admission counts

* 30-day discharge counts

In [10]:
admit_url = 'https://data.cityofnewyork.us/resource/6teu-xtgp.json'
dis_url = 'https://data.cityofnewyork.us/resource/94ri-3ium.json'
first_st_date_adp = _30day_pop_tot.iloc[0]['Start Date'].date()
_30day_admit_df = get_agg_admit_dis_data(first_st_date_adp, admit_url, 'ADMITTED_DT',30)
_30day_dis_df = get_agg_admit_dis_data(first_st_date_adp, dis_url, 'DISCHARGED_DT',30)

In [11]:
_30day_admit_df.head()

,Start Date,admission_count,End Date,Month,Year,Days to Max Date
0,2016-05-15,4935,2016-06-13,5,2016,3030
1,2016-06-14,4897,2016-07-13,6,2016,3000
2,2016-07-14,4871,2016-08-12,7,2016,2970
3,2016-08-13,4600,2016-09-11,8,2016,2940
4,2016-09-12,4519,2016-10-11,9,2016,2910


In [12]:
_30day_admit_df.tail()

,Start Date,admission_count,End Date,Month,Year,Days to Max Date
96,2024-04-03,1961,2024-05-02,4,2024,150
97,2024-05-03,1979,2024-06-01,5,2024,120
98,2024-06-02,2011,2024-07-01,6,2024,90
99,2024-07-02,1903,2024-07-31,7,2024,60
100,2024-08-01,2005,2024-08-30,8,2024,30


In [13]:
_30day_dis_df.head()

,Start Date,discharge_count,End Date,Month,Year,Days to Max Date
0,2016-05-15,4880,2016-06-13,5,2016,3030
1,2016-06-14,4999,2016-07-13,6,2016,3000
2,2016-07-14,4874,2016-08-12,7,2016,2970
3,2016-08-13,4338,2016-09-11,8,2016,2940
4,2016-09-12,4666,2016-10-11,9,2016,2910


In [14]:
_30day_dis_df.tail()

,Start Date,discharge_count,End Date,Month,Year,Days to Max Date
96,2024-04-03,1947,2024-05-02,4,2024,150
97,2024-05-03,1896,2024-06-01,5,2024,120
98,2024-06-02,1962,2024-07-01,6,2024,90
99,2024-07-02,1907,2024-07-31,7,2024,60
100,2024-08-01,1975,2024-08-30,8,2024,30


In [15]:
#test los function
_30_day_los_df = get_los_data(first_st_date_adp,30)
_30_day_los_df.head()

,Start Date,Avg LOS Days,End Date,Discharge Month,Discharge Year,Days to Max Date
0,2016-05-15,56.661485,2016-06-13,5,2016,3030
1,2016-06-14,62.825000,2016-07-13,6,2016,3000
2,2016-07-14,61.316523,2016-08-12,7,2016,2970
3,2016-08-13,56.608826,2016-09-11,8,2016,2940
4,2016-09-12,61.751337,2016-10-11,9,2016,2910


In [16]:
_30_day_los_df.tail()

,Start Date,Avg LOS Days,End Date,Discharge Month,Discharge Year,Days to Max Date
96,2024-04-03,102.004620,2024-05-02,4,2024,150
97,2024-05-03,100.445205,2024-06-01,5,2024,120
98,2024-06-02,101.879004,2024-07-01,6,2024,90
99,2024-07-02,93.435992,2024-07-31,7,2024,60
100,2024-08-01,96.664470,2024-08-30,8,2024,30


In [17]:
#join to one dataframe and store as csv
_30day_joined_ivs_df = _30day_admit_df[['Start Date','End Date','admission_count']].merge(_30day_dis_df[['Start Date','End Date','discharge_count']],
                                                                                left_on = ['Start Date','End Date'], right_on = ['Start Date','End Date'])
_30day_joined_ivs_df = _30day_joined_ivs_df.merge(_30_day_los_df[['Start Date','End Date','Avg LOS Days']],
                                    left_on = ['Start Date','End Date'], right_on = ['Start Date','End Date'])
_30day_joined_ivs_df.head()

,Start Date,End Date,admission_count,discharge_count,Avg LOS Days
0,2016-05-15,2016-06-13,4935,4880,56.661485
1,2016-06-14,2016-07-13,4897,4999,62.825000
2,2016-07-14,2016-08-12,4871,4874,61.316523
3,2016-08-13,2016-09-11,4600,4338,56.608826
4,2016-09-12,2016-10-11,4519,4666,61.751337


In [18]:
_30day_joined_ivs_df.tail()

,Start Date,End Date,admission_count,discharge_count,Avg LOS Days
96,2024-04-03,2024-05-02,1961,1947,102.004620
97,2024-05-03,2024-06-01,1979,1896,100.445205
98,2024-06-02,2024-07-01,2011,1962,101.879004
99,2024-07-02,2024-07-31,1903,1907,93.435992
100,2024-08-01,2024-08-30,2005,1975,96.664470


In [19]:
#save to csv
_30day_joined_ivs_df.to_csv("../Data/_30_day_IVs.csv")

In [20]:
#repeat for 7 day
_7day_admit_df = get_agg_admit_dis_data(first_st_date_adp, admit_url, 'ADMITTED_DT',7)
_7day_dis_df = get_agg_admit_dis_data(first_st_date_adp, dis_url, 'DISCHARGED_DT',7)
_7day_los_df = get_los_data(first_st_date_adp,7)
#merge
_7day_joined_ivs_df = _7day_admit_df[['Start Date','End Date','admission_count']].merge(_7day_dis_df[['Start Date','End Date','discharge_count']],
                                                                                left_on = ['Start Date','End Date'], right_on = ['Start Date','End Date'])
_7day_joined_ivs_df = _7day_joined_ivs_df.merge(_7day_los_df[['Start Date','End Date','Avg LOS Days']],
                                    left_on = ['Start Date','End Date'], right_on = ['Start Date','End Date'])
_7day_joined_ivs_df.head()


,Start Date,End Date,admission_count,discharge_count,Avg LOS Days
0,2016-05-15,2016-05-21,1280,1206,52.026512
1,2016-05-22,2016-05-28,1174,1305,63.443252
2,2016-05-29,2016-06-04,1105,1051,56.146110
3,2016-06-05,2016-06-11,1102,1124,53.384206
4,2016-06-12,2016-06-18,1157,1155,59.323529


In [21]:
#save to csv
_7day_joined_ivs_df.to_csv("../Data/_7_day_IVs.csv")

In [22]:
#test complaint function
crime_data = get_crime_data(first_st_date_adp,30)
crime_data.head()

,Start Date,total_felony_crimes,violent_felony_crimes,total_misdemeanor_crimes,murder_homicide_count,robbery_count,assault_count,burglary_count,rape_count,grand_larceny_count,grand_larceny_vehicle_count,weapons_count,End Date,Days to Max Date
0,2016-05-15,12504,3330,23148,30,1284,1876,1053,140,3651,585,466,2016-06-13,2968
1,2016-06-14,12792,3422,22376,32,1293,1958,1072,139,3795,559,418,2016-07-13,2938
2,2016-07-14,13079,3588,22312,46,1354,2046,1128,142,3855,628,379,2016-08-12,2908
3,2016-08-13,13320,3530,22284,37,1364,2001,1116,128,3805,657,506,2016-09-11,2878
4,2016-09-12,12309,3160,21337,33,1338,1674,1084,115,3621,561,444,2016-10-11,2848


In [23]:
crime_data.tail()

,Start Date,total_felony_crimes,violent_felony_crimes,total_misdemeanor_crimes,murder_homicide_count,robbery_count,assault_count,burglary_count,rape_count,grand_larceny_count,grand_larceny_vehicle_count,weapons_count,End Date,Days to Max Date
93,2024-01-04,14510,3507,23096,23,1400,1998,1057,86,3836,1080,451,2024-02-02,178
94,2024-02-03,14056,3491,23436,21,1257,2115,1062,98,3552,950,419,2024-03-03,148
95,2024-03-04,14166,3599,22936,29,1255,2216,994,99,3605,1047,455,2024-04-02,118
96,2024-04-03,14550,3812,23505,20,1339,2364,1025,89,3490,1090,450,2024-05-02,88
97,2024-05-03,15032,4169,24261,31,1425,2617,1034,96,3552,1178,494,2024-06-01,58


In [24]:
_30day_pop_tot.head(1)

,Start Date,End Date,ADP
0,2016-05-15,2016-06-13,9820.0


In [25]:
_30day_pop_tot.tail(1)

,Start Date,End Date,ADP
101,2024-08-31,2024-09-29,6509.0


In [26]:
crime_data.columns[1:-2].to_list()

['total_felony_crimes',
 'violent_felony_crimes',
 'total_misdemeanor_crimes',
 'murder_homicide_count',
 'robbery_count',
 'assault_count',
 'burglary_count',
 'rape_count',
 'grand_larceny_count',
 'grand_larceny_vehicle_count',
 'weapons_count']

In [27]:
print(['Start Date', 'End Date']+crime_data.columns[1:-2].to_list())

['Start Date', 'End Date', 'total_felony_crimes', 'violent_felony_crimes', 'total_misdemeanor_crimes', 'murder_homicide_count', 'robbery_count', 'assault_count', 'burglary_count', 'rape_count', 'grand_larceny_count', 'grand_larceny_vehicle_count', 'weapons_count']


In [28]:
crime_data = crime_data[['Start Date', 'End Date']+crime_data.columns[1:-2].to_list()]
crime_data['nonviolent_felony_crimes'] = crime_data['total_felony_crimes'] - crime_data['violent_felony_crimes']
print(crime_data.columns)

Index(['Start Date', 'End Date', 'total_felony_crimes',
       'violent_felony_crimes', 'total_misdemeanor_crimes',
       'murder_homicide_count', 'robbery_count', 'assault_count',
       'burglary_count', 'rape_count', 'grand_larceny_count',
       'grand_larceny_vehicle_count', 'weapons_count',
       'nonviolent_felony_crimes'],
      dtype='object')


In [29]:
crime_data.head()

,Start Date,End Date,total_felony_crimes,violent_felony_crimes,total_misdemeanor_crimes,murder_homicide_count,robbery_count,assault_count,burglary_count,rape_count,grand_larceny_count,grand_larceny_vehicle_count,weapons_count,nonviolent_felony_crimes
0,2016-05-15,2016-06-13,12504,3330,23148,30,1284,1876,1053,140,3651,585,466,9174
1,2016-06-14,2016-07-13,12792,3422,22376,32,1293,1958,1072,139,3795,559,418,9370
2,2016-07-14,2016-08-12,13079,3588,22312,46,1354,2046,1128,142,3855,628,379,9491
3,2016-08-13,2016-09-11,13320,3530,22284,37,1364,2001,1116,128,3805,657,506,9790
4,2016-09-12,2016-10-11,12309,3160,21337,33,1338,1674,1084,115,3621,561,444,9149


In [30]:
#save to csv
crime_data.to_csv("../Data/_30_day_crime_counts.csv")

In [31]:
arrest_data = get_arrest_data(first_st_date_adp, 30)
arrest_data.head()

,Start Date,total_felony_arrest,violent_felony_arrest,total_misdemeanor_arrest,arrest_murder_homicide_count,arrest_robbery_count,arrest_assault_count,arrest_burglary_count,arrest_rape_count,arrest_grand_larceny_count,arrest_grand_larceny_vehicle_count,arrest_weapons_count,End Date,Days to Max Date
0,2016-05-15,7890,2353,17655,65,868,1349,402,71,852,89,614,2016-06-13,2968
1,2016-06-14,7647,2332,16604,89,785,1391,325,67,836,104,568,2016-07-13,2938
2,2016-07-14,7849,2465,15893,101,878,1409,385,77,815,85,499,2016-08-12,2908
3,2016-08-13,7812,2334,16682,79,826,1359,423,70,803,104,643,2016-09-11,2878
4,2016-09-12,7582,2187,16690,76,885,1168,426,58,767,114,565,2016-10-11,2848


In [32]:
arrest_data.tail(1)

,Start Date,total_felony_arrest,violent_felony_arrest,total_misdemeanor_arrest,arrest_murder_homicide_count,arrest_robbery_count,arrest_assault_count,arrest_burglary_count,arrest_rape_count,arrest_grand_larceny_count,arrest_grand_larceny_vehicle_count,arrest_weapons_count,End Date,Days to Max Date
97,2024-05-03,8977,3008,12049,4,989,1954,558,61,999,172,619,2024-06-01,58


In [33]:
arrest_data = arrest_data[['Start Date', 'End Date']+arrest_data.columns[1:-2].to_list()]
arrest_data['nonviolent_felony_arrest'] = arrest_data['total_felony_arrest'] - arrest_data['violent_felony_arrest']
print(arrest_data.columns)

Index(['Start Date', 'End Date', 'total_felony_arrest',
       'violent_felony_arrest', 'total_misdemeanor_arrest',
       'arrest_murder_homicide_count', 'arrest_robbery_count',
       'arrest_assault_count', 'arrest_burglary_count', 'arrest_rape_count',
       'arrest_grand_larceny_count', 'arrest_grand_larceny_vehicle_count',
       'arrest_weapons_count', 'nonviolent_felony_arrest'],
      dtype='object')


In [34]:
#save to csv
arrest_data.to_csv("../Data/_30_day_arrest_counts.csv")

In [35]:
#repeat for 7 day
_7day_crime_data = get_crime_data(first_st_date_adp,7)
_7day_arrest_data = get_arrest_data(first_st_date_adp,7)

#save to csv
_7day_crime_data[['Start Date', 'End Date']+crime_data.columns[1:-2].to_list()].to_csv("../Data/_7_day_crime_counts.csv")
_7day_arrest_data[['Start Date', 'End Date']+arrest_data.columns[1:-2].to_list()].to_csv("../Data/_7_day_arrest_counts.csv")